In [1]:
%pip install torch transformers sentence-transformers numpy


   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   -- ------------------------------------- 10.2/171.5 kB ? eta -:--:--
   --------- ----------------------------- 41.0/171.5 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 171.5/171.5 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install faiss-cpu

In [11]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss


In [3]:
# Load the pre-trained GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [4]:
# Load the SentenceTransformer model for encoding documents
sentence_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Load the dataset and build the FAISS index
def build_index(documents):
    document_embeddings = sentence_model.encode(documents, convert_to_tensor=True)
    index = faiss.IndexFlatIP(document_embeddings.size(-1))
    index.add(document_embeddings.numpy())
    return index

In [29]:
def retrieve_documents(index, query, documents, k=3):
    query_embedding = sentence_model.encode(query, convert_to_tensor=True)
    if len(query_embedding.shape) == 2:  # Ensure query_embedding is 2-dimensional
        D, I = index.search(query_embedding.numpy(), k)
        print("Distances:", D)
        print("Indices:", I)
        if isinstance(D, np.ndarray) and isinstance(I, np.ndarray):
            return [documents[i] for i in I[0]]
    else:
        print("Error: Query embedding shape is not as expected")
    return []


In [30]:
# Example retrieved documents (you'd replace this with actual retrieval logic)
# Assume 'states.txt' contains documents about different states
with open("states.txt", "r", encoding="utf-8") as file:
    documents = file.readlines()

In [31]:
# Build the FAISS index
index = build_index(documents)

In [32]:
# Loop to take user queries until 'quit' is entered
while True:
    # Take user query
    user_query = input("Please enter your question (type 'quit' to exit): ")
    if user_query.lower() == 'quit':
        break

    # Retrieve relevant documents
    retrieved_documents = retrieve_documents(index, user_query, documents)

    # Combine user query and retrieved documents
    response = user_query + "\n" + "\n".join(retrieved_documents)

    # Generate response using GPT-2 model
    input_ids = tokenizer.encode(response, return_tensors="pt", max_length=1024, truncation=True)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)

    # Decode and print the response
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Generated Response:")
    print(generated_response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error: Query embedding shape is not as expected
Generated Response:
what is delhi?

Delhi is a Sanskrit word meaning "to be" or "in the presence of". It is also used in the sense of "the presence" of a person or thing.
. It means "a person, thing, or object".
 (Deli) is the Sanskrit term for "thing". The word is used to refer to a thing or a place. The term is often used as a synonym for a "person". (Dharma


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error: Query embedding shape is not as expected
Generated Response:
what is mumbai called as?

Mumbai is a city of people, people of colour, and people who are not afraid to speak their mind.
...
,
:
The people are the ones who speak the truth. The people speak truth because they are afraid of being misunderstood. They are scared of the people. And they speak it because it is their truth that they want to hear. It is the only way to be heard. If you are a person of


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error: Query embedding shape is not as expected
Generated Response:
what is chennai?

The answer is that the city is a city of the people.
...
 (The people are the ones who are in charge of this city.)
,
- The people who live in the cities are called the "people of God."
: The city that is called "the city" is the one that has been created by God. The "city" that was created is God's city. God created the world. He created man


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Error: Query embedding shape is not as expected
Generated Response:
where is kolkata?

Kolkat: I am not sure. I have not seen it.
 (Laughs)
...
,.,. (laughs)...
:. I don't know. It's not like I'm going to go to the airport. But I will go. (Laughter) I'll go, I promise. And I won't go back. So I can't say anything. No, no, it's fine.
